In [58]:
import utilities
from utilities import *
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
import os

def test_sentence(sentence):
    sentence = process_text(sentence)
    vec = vectorizer.transform([sentence]).toarray()
    prediction = model.predict_proba(vec)[0]
    print "Label", label_names[np.argmax(prediction)]
    for i, category in enumerate(label_names):
        print "{}: {:.2f}".format(category, prediction[i])

# Constants
no_dev = True
binary = False

In [67]:
print "Get and process data"
# Get raw text + labels
features, labels, label_names = get_data('readme', binary=binary, no_dev=no_dev)
features2, labels2, label_names2 = get_data('description', binary=binary, no_dev=no_dev)
features = features + features2
labels = labels + labels2
features, labels = shuffle_data(features, labels)
#features = np.matrix(features)

print "Vectorize data"
# trainingsdaten werden in eingabedaten (vektoren) umgewandelt
# features ist dann matrix bestehend aus den einzelnen vektore
features, vectorizer = vectorize_text(features, max_features=3000)

# x sind die eingabematrizen, y sind die vektoren in denen die ergebnisse stehen
x_train, x_test, y_train, y_test = split_train_test(features, labels, ratio=0.7, shuffle=True)

print "Prepare one-hot-encoding"
# One-Hot-Encoding needed for Neural Net Output
y_train = one_hot_encoding(y_train)
y_test = one_hot_encoding(y_test)

Get and process data
Vectorize data
Prepare one-hot-encoding


In [68]:
print "Build model"
model = None
trained_model_filneame = 'modelnondev'

if trained_model_filneame not in os.listdir('.'):
    # Struktur des Netzes
    model = Sequential()
    input_size = x_train.shape[1]
    model.add(Dense(input_size, input_dim=input_size))
    model.add(Activation('relu'))
    model.add(Dense(input_size * 3))
    model.add(Activation('relu'))
    model.add(Dense(input_size * 3)) # Let's make it deeeep
    model.add(Activation('relu'))
    # Output Layer, one neuron per class
    if binary:
        model.add(Dense(2))
    else:
        if no_dev:
            model.add(Dense(7))
        else:
            model.add(Dense(8))
    # Softmax zum Normalisieren der Werte, damit Wert des Neurons WSK in % angibt
    model.add(Activation('softmax'))

    model.compile(metrics=['accuracy'], optimizer='adam', loss='categorical_crossentropy')
else:
    model = keras.models.load_model(trained_model_filneame)

Build model


In [70]:
print "Train model"
model.fit(x_train, y_train, nb_epoch=10, shuffle=True, batch_size=8)

Train model
Epoch 1/10
443/443 [==============================] - 97s - loss: 1.1246 - acc: 0.6117    
Epoch 2/10
443/443 [==============================] - 105s - loss: 0.6502 - acc: 0.7810   
Epoch 3/10
443/443 [==============================] - 103s - loss: 0.5256 - acc: 0.8217   
Epoch 4/10
443/443 [==============================] - 102s - loss: 0.5255 - acc: 0.8172   
Epoch 5/10
443/443 [==============================] - 104s - loss: 0.4239 - acc: 0.8510   
Epoch 6/10
443/443 [==============================] - 108s - loss: 0.3980 - acc: 0.8600   
Epoch 7/10
443/443 [==============================] - 109s - loss: 0.3837 - acc: 0.8623   
Epoch 8/10
443/443 [==============================] - 102s - loss: 0.4117 - acc: 0.8646   
Epoch 9/10
443/443 [==============================] - 111s - loss: 0.4219 - acc: 0.8646   
Epoch 10/10
443/443 [==============================] - 123s - loss: 0.4148 - acc: 0.8646   


In [52]:
# Get accuracy on test_set
print "Test on {} unknown samples".format(len(x_test))
acc = model.evaluate(x_test, y_test, verbose=0)
print "Loss, accuracy: ", acc

Test on 95 unknown samples
Loss, accuracy:  [0.70906480864474652, 0.84210525562888694]


In [66]:
# Test model on some sentence
# 
test_sentence('')

1/1 [==============================] - 0s
Label OTHER
WEB: 0.21
DOCS: 0.16
OTHER: 0.37
EDU: 0.03
HW: 0.03
DATA: 0.10
UNSURE: 0.10


In [ ]:
# Save model
model.save(trained_model_filneame)